In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Function to calculate daily returns
def calculate_returns(prices):
    returns = prices.pct_change().dropna()
    return returns

# Function to calculate volatility (risk assessment)
def calculate_volatility(returns):
    volatility = returns.std()  # Standard deviation of returns (volatility)
    return volatility

# Function to calculate expected return
def calculate_expected_return(returns):
    expected_return = returns.mean()  # Average daily return
    return expected_return

# Function to calculate the probability of losing money
def probability_of_losing(returns):
    mean_return = returns.mean()
    std_return = returns.std()

    # Using normal distribution, calculate probability of returns less than 0
    prob_losing = norm.cdf(0, mean_return, std_return)
    return prob_losing

# Function to calculate the probability of winning (return > 0)
def probability_of_winning(returns):
    return 1 - probability_of_losing(returns)

# Main function to calculate risk, expected return, and probabilities
def risk_and_probability_analysis(prices):
    returns = calculate_returns(prices)
    
    # Risk Assessment
    volatility = calculate_volatility(returns)
    
    # Expected Return
    expected_return = calculate_expected_return(returns)
    
    # Probability of Losing
    prob_losing = probability_of_losing(returns)
    
    # Probability of Winning
    prob_winning = probability_of_winning(returns)
    
    return {
        "Volatility (Risk)": volatility,
        "Expected Return": expected_return,
        "Probability of Losing": prob_losing,
        "Probability of Winning": prob_winning
    }

# Example usage with dummy historical prices
# Suppose we have a dataframe of daily closing prices for a stock
data = {
    "Date": pd.date_range(start="2023-01-01", periods=10, freq='D'),
    "Price": [100, 102, 101, 105, 108, 107, 110, 111, 115, 114]
}
df = pd.DataFrame(data)
prices = df["Price"]

# Perform risk and probability analysis
result = risk_and_probability_analysis(prices)

# Print results
print("Risk and Probability Analysis:")
for key, value in result.items():
    print(f"{key}: {value:.4f}")

Risk and Probability Analysis:
Volatility (Risk): 0.0201
Expected Return: 0.0148
Probability of Losing: 0.2300
Probability of Winning: 0.7700


In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# 1. Probability of Drawdown
def calculate_drawdown(prices):
    # Calculate the cumulative max of the stock prices
    cumulative_max = prices.cummax()
    
    # Calculate drawdown as the percentage decline from the peak
    drawdown = (prices - cumulative_max) / cumulative_max
    return drawdown

# Probability of Drawdown exceeding a threshold
def probability_of_drawdown(prices, threshold):
    drawdown = calculate_drawdown(prices)
    prob_drawdown = (drawdown <= -threshold).mean()  # Probability of drawdown exceeding threshold
    return prob_drawdown

# 2. Probability of Recovery from a Drawdown
def probability_of_recovery(prices, recovery_periods):
    drawdown = calculate_drawdown(prices)
    
    # Define the recovery as returning to the previous peak (drawdown == 0)
    recovery = (drawdown == 0).rolling(window=recovery_periods).sum() > 0
    prob_recovery = recovery.mean()
    
    return prob_recovery

# 3. Bayesian Analysis Probability
# Prior, likelihood, and updating the probability using Bayes' Theorem
def bayesian_update(prior, likelihood_positive, likelihood_negative, evidence_positive):
    # P(H|E) = P(E|H) * P(H) / P(E)
    posterior_positive = (likelihood_positive * prior) / evidence_positive
    posterior_negative = (likelihood_negative * (1 - prior)) / (1 - evidence_positive)
    
    # Normalizing to ensure probabilities sum to 1
    normalization_factor = posterior_positive + posterior_negative
    posterior_positive /= normalization_factor
    posterior_negative /= normalization_factor
    
    return posterior_positive, posterior_negative

# Main function
def stock_analysis(prices, drawdown_threshold, recovery_periods, prior, likelihood_positive, likelihood_negative, evidence_positive):
    # Calculate Probability of Drawdown
    prob_drawdown = probability_of_drawdown(prices, drawdown_threshold)
    
    # Calculate Probability of Recovery from Drawdown
    prob_recovery = probability_of_recovery(prices, recovery_periods)
    
    # Bayesian Analysis for future stock movement
    posterior_positive, posterior_negative = bayesian_update(prior, likelihood_positive, likelihood_negative, evidence_positive)
    
    return {
        "Probability of Drawdown": prob_drawdown,
        "Probability of Recovery": prob_recovery,
        "Posterior Positive (Bayesian)": posterior_positive,
        "Posterior Negative (Bayesian)": posterior_negative
    }

# Example usage
# Suppose we have a dataframe of daily closing prices for a stock
data = {
    "Date": pd.date_range(start="2023-01-01", periods=10, freq='D'),
    "Price": [100, 105, 101, 110, 108, 112, 111, 120, 119, 115]
}
df = pd.DataFrame(data)
prices = df["Price"]

# Parameters for Bayesian Analysis
prior = 0.5  # Initial belief of a positive outcome
likelihood_positive = 0.7  # Likelihood of evidence given a positive outcome
likelihood_negative = 0.3  # Likelihood of evidence given a negative outcome
evidence_positive = 0.6  # Evidence in favor of a positive outcome

# Perform analysis
result = stock_analysis(prices, drawdown_threshold=0.05, recovery_periods=3, prior=prior,
                        likelihood_positive=likelihood_positive, likelihood_negative=likelihood_negative,
                        evidence_positive=evidence_positive)

# Print results
print("Stock Analysis Results:")
for key, value in result.items():
    print(f"{key}: {value:.4f}")

Stock Analysis Results:
Probability of Drawdown: 0.0000
Probability of Recovery: 0.8000
Posterior Positive (Bayesian): 0.6087
Posterior Negative (Bayesian): 0.3913
